## Test models to ensure that the same results are achieved when training with the same seeds

In [1]:
%load_ext autoreload
%autoreload 2
# this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

from cleanplots import *
from tqdm import tqdm
from encoding_information.information_estimation import *
from encoding_information.image_utils import *
from encoding_information.bsccm_utils import *
from bsccm import BSCCM
from jax import jit
import numpy as onp
import jax.numpy as np

bsccm = BSCCM('/home/hpinkard_waller/data/BSCCM/')

2024-09-06 13:49:29.132704: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 13:49:29.787501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-09-06 13:49:29.787589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-09-06 13:49:29.787597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Opening BSCCM
Opened BSCCM


In [2]:
channel_name = 'LED119'
num_images = 1000
num_patches = 1000
patch_size = 15
edge_crop = 24

noise_seed = 1234
patches_seed = 1234567

images = load_bsccm_images(bsccm, channel_name, num_images=num_images, 
                            edge_crop=edge_crop, convert_units_to_photons=True, median_filter=False, verbose=False, batch=1)
patches = extract_patches(images, patch_size=patch_size, num_patches=num_patches, verbose=False, seed=patches_seed)

noisy_patches = add_shot_noise_to_experimenal_data(patches, 0.2, seed=noise_seed)

train_set = noisy_patches[:int(0.8*len(noisy_patches))]
test_set = noisy_patches[int(0.8*len(noisy_patches)):]


## PixelCNN

In [3]:
# Train two models with the same seeds, and on model with a different seed, and compare the NLLs
model_seed = 4321 # Initialize weights
data_seed = 81 # initialize data shuffling and noise addition

noisy_image_model1 = PixelCNN()
noisy_image_model1.fit(train_set, verbose=True, model_seed=model_seed, data_seed=data_seed)
nll1 = noisy_image_model1.compute_negative_log_likelihood(test_set, verbose=True)
nll1_again = noisy_image_model1.compute_negative_log_likelihood(test_set, verbose=True)

noisy_image_model2 = PixelCNN()
noisy_image_model2.fit(train_set, verbose=True, model_seed=model_seed, data_seed=data_seed)
nll2 = noisy_image_model2.compute_negative_log_likelihood(test_set, verbose=True)
nll2_again = noisy_image_model2.compute_negative_log_likelihood(test_set, verbose=True)


model_seed = 1234
data_seed = 84

noisy_image_model3 = PixelCNN()
noisy_image_model3.fit(train_set, verbose=True, model_seed=model_seed, data_seed=data_seed)
nll3 = noisy_image_model3.compute_negative_log_likelihood(test_set, verbose=True)

should_be_same = abs(nll1 - nll2)
should_be_different = abs(nll1 - nll3)

print(f"Should be same: {should_be_same}")
print(f"Should be different: {should_be_different}")


2024-09-06 11:38:04.719675: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Initial validation NLL: 24.64


Epoch 1: 100%|██████████| 100/100 [00:05<00:00, 17.05it/s]


Epoch 1: validation NLL: 4.52


Epoch 2: 100%|██████████| 100/100 [00:01<00:00, 60.02it/s]


...

Epoch 41: validation NLL: 5.76


Epoch 42: 100%|██████████| 100/100 [00:01<00:00, 58.61it/s]


Epoch 42: validation NLL: 5.65


Epoch 43: 100%|██████████| 100/100 [00:01<00:00, 61.98it/s]


Epoch 43: validation NLL: 5.77
Should be same: 0.003098011016845703
Should be different: 0.0003681182861328125


## Full Gaussian

In [4]:
# Train two models with the same seeds, and on model with a different seed, and compare the NLLs
data_seed = 81
epochs = 1
steps_per_epoch = 5

noisy_image_model = FullGaussianProcess(train_set, seed=data_seed)
# (no fit needed)
nll1 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)

noisy_image_model = FullGaussianProcess(train_set, seed=data_seed)
# (no fit needed)
nll2 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)


data_seed = 84

noisy_image_model = FullGaussianProcess(train_set, seed=data_seed)
# (no fit needed)
nll3 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)


should_be_same = abs(nll1 - nll2)
should_be_different = abs(nll1 - nll3)

print(f"Should be same: {should_be_same}")
print(f"Should be different: {should_be_different}")


2024-09-06 13:50:14.381186: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Should be same: 0.0
Should be different: 4.482269287109375e-05


## Stationary Gaussian

In [6]:
# Train two models with the same seeds, and on model with a different seed, and compare the NLLs
data_seed = 81
epochs = 1
steps_per_epoch = 5

noisy_image_model = StationaryGaussianProcess(train_set, seed=data_seed)
noisy_image_model.fit(data_seed=data_seed)
nll1 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)

noisy_image_model = StationaryGaussianProcess(train_set, seed=data_seed)
noisy_image_model.fit(data_seed=data_seed)
nll2 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)


model_seed = 1234
data_seed = 84

noisy_image_model = StationaryGaussianProcess(train_set, seed=data_seed)
noisy_image_model.fit(data_seed=data_seed)
nll3 = noisy_image_model.compute_negative_log_likelihood(test_set, seed=data_seed, verbose=True)


should_be_same = abs(nll1 - nll2)
should_be_different = abs(nll1 - nll3)

print(f"Should be same: {should_be_same}")
print(f"Should be different: {should_be_different}")


Initial validation NLL: 4.41


Epoch 1: 100%|██████████| 1/1 [00:06<00:00,  6.77s/it]


Epoch 1: validation NLL: 4.41


Epoch 2: 100%|██████████| 1/1 [00:00<00:00, 32.29it/s]


Epoch 2: validation NLL: 4.41


...

Epoch 43: validation NLL: 4.41


precomputing masks and variances: 100%|██████████| 15/15 [00:00<00:00, 150.26it/s]


evaluating likelihood


computing log likelihoods: 100%|██████████| 15/15 [00:00<00:00, 141.16it/s]

Should be same: 0.0
Should be different: 7.606027956530426e-05
